In [ ]:
# Import necessary libraries

import numpy as np
import pandas as pd
from pandas_datareader import data as pdr
from datetime import datetime as dt
import yfinance as yf

# Enable Yahoo Finance override for pandas_datareader
yf.pdr_override()

# Define time period for analysis
start = dt(2012, 1, 1)
end = dt(2016, 12, 31)

# Define tickers: PG (Procter & Gamble) and S&P500 index (^GSPC as market proxy)
tickers = ['PG', '^GSPC']

# Initialize empty DataFrame
data = pd.DataFrame()

# Download adjusted close prices for each ticker and store in DataFrame
for t in tickers :
    data[t] = pdr.DataReader(t, start, end)['Adj Close']

# Display few rows of stock prices
data

[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed


,PG,^GSPC
Date,,
2012-01-03,47.025040,1277.060059
2012-01-04,47.003933,1277.300049
2012-01-05,46.806911,1281.060059
2012-01-06,46.694321,1277.810059
2012-01-09,46.891346,1280.699951
...,...,...
2016-12-23,70.117630,2263.790039
2016-12-27,69.820496,2268.879883
2016-12-28,69.383087,2249.919922


In [ ]:
# Compute daily log returns for each asset
# log return = ln(Pt / Pt-1)

sec_return = np.log(data / data.shift(1))
sec_return

,PG,^GSPC
Date,,
2012-01-03,NaN,NaN
2012-01-04,-0.000449,0.000188
2012-01-05,-0.004200,0.002939
2012-01-06,-0.002408,-0.002540
2012-01-09,0.004211,0.002259
...,...,...
2016-12-23,0.005785,0.001251
2016-12-27,-0.004247,0.002246
2016-12-28,-0.006284,-0.008392


In [ ]:
# Calculate the average daily return of PG

avg_return = sec_return.mean()['PG']
avg_return

0.0003095319556891509

In [ ]:
# Compute annualized covariance matrix (multiplied by 250 trading days)

cov = sec_return.cov()*250
cov

,PG,^GSPC
PG,0.020409,0.010078
^GSPC,0.010078,0.016362


In [ ]:
# Extract covariance of PG with the market (S&P500)

cov_with_market = cov.iloc[0,1]
cov_with_market

0.010078075720957837

In [ ]:
# Calculate annualized variance of the market (S&P500)

market_var = sec_return['^GSPC'].var() * 250
market_var

0.016361631002308474

## BETA

In [ ]:
# --- BETA Calculation ---
# Beta = Cov(PG, Market) / Var(Market)

PG_beta = cov_with_market / market_var
PG_beta

0.6159578907222584

In [ ]:
# Define risk-free rate (rf) and market risk premium (rm - rf)

rf = 0.025         # 2.5% annual risk-free rate
rm_rf = 0.05       # 5% market risk premium

In [ ]:
# CAPM: Expected Return of PG = rf + Beta * (rm - rf)

PG_er = rf + PG_beta * rm_rf
PG_er

0.05579789453611292

In [ ]:
# --- Sharpe Ratio Calculation ---
# Sharpe = (Expected Return - Risk-free Rate) / Volatility

sharpe = (PG_er - rf) / (sec_return['PG'].std() * 250 ** 0.5)
sharpe

0.2155808839460718